# Stats Practice3

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import scipy.stats as stats
import sklearn as sk
import matplotlib.pyplot as plt

In [2]:
import sys
print(sys.version)

3.7.8 (tags/v3.7.8:4b47a5b6ba, Jun 28 2020, 08:53:46) [MSC v.1916 64 bit (AMD64)]


## 통계를 배우는 목적
1. 집단(모집단, 표본)에 대한 정량적 이해 – 분포, 대표값(점 추정, 구간 추정)
2. 의사 결정 – 검정 (표본의 모집단내 위치에 기반), 임계값, 유의 확률
3. 비교 – 표준값
4. 관련성(Relation) – 상관계수(Correlation), 상관비, 연관 계수(Association)
5. 예측 - 회귀

### [예제 1] '비료의 종류'와 '수확량'의 관계 (<u>일원배치 분산분석</u>)
출처 : 쿠리하라 신이치, 통계학 도감, 2019, pp. 105

In [3]:
one_way_anova = pd.DataFrame([[4, 13, 22], [6, 9, 18]], \
                        index=['1회', '2회'], columns=['비료 없음', '비료 A', '비료 B'])
one_way_anova

,비료 없음,비료 A,비료 B
1회,4,13,22
2회,6,9,18


In [4]:
# 표에 다음을 추가한다
one_way_anova.loc['회수'] = one_way_anova.iloc[0:2].count(axis=0)
one_way_anova.loc['합계'] = one_way_anova.iloc[0:2].sum(axis=0)
one_way_anova.loc['평균'] = one_way_anova.iloc[0:2].mean(axis=0)

In [5]:
one_way_anova

,비료 없음,비료 A,비료 B
1회,4.0,13.0,22.0
2회,6.0,9.0,18.0
회수,2.0,2.0,2.0
합계,10.0,22.0,40.0
평균,5.0,11.0,20.0


In [6]:
# SSB(Sum of Squares Between)를 계산한다
sum_between = one_way_anova[one_way_anova.index=='합계'].sum(axis=1)[0]
count_between = one_way_anova[one_way_anova.index=='회수'].sum(axis=1)[0]
sum_between, count_between
mean_between = sum_between / count_between

cols = one_way_anova.shape[1]
ss_between = 0
irow_count = 2
irow_mean = 4
for icol in range(0, cols):
    ss_between = ss_between + pow(one_way_anova.iat[irow_mean, icol] - mean_between, 2) * one_way_anova.iat[irow_count, icol]

df_between = 3 - 1 # 수준 : 3 
ms_between = ss_between / df_between

print('ss_between = ', round(ss_between, 2), ', df_between = ', df_between, ', ms_between = ', round(ms_between, 2))

ss_between =  228.0 , df_between =  2 , ms_between =  114.0


In [7]:
# SSW(Sum of Squares Within)를 계산한다
one_way_anova_ss = one_way_anova.iloc[0:2].copy() # 깊은 복사
one_way_anova_ss = one_way_anova_ss.astype('float') # 데이터 타입 변경

def calcSS(one_way_anova, one_way_anova_ss):
    idx_col_sum = one_way_anova.shape[1]
    idx_row_sum = one_way_anova.shape[0] - 1
    for irow in range(0, idx_row_sum - 2):
        for icol in range(0, idx_col_sum):
            one_way_anova_ss.iat[irow, icol] = pow(one_way_anova.iat[irow, icol] - one_way_anova.iat[idx_row_sum, icol], 2)

calcSS(one_way_anova, one_way_anova_ss)

# 표에 SS 합계를 추가한다
one_way_anova_ss.loc['SS 합계'] = one_way_anova_ss.iloc[0:2].sum(axis=0)
        
ss_within = one_way_anova_ss.iloc[-1:].sum(axis=1)[0]

df_within = 6 - 3 # 데이터 수(6) - 수준의 수(3)
ms_within = ss_within / df_within

print('ss_within = ', round(ss_within, 2), ', df_within = ', df_within, ', ms_within = ', round(ms_within, 2))

ss_within =  18.0 , df_within =  3 , ms_within =  6.0


In [8]:
# 가설을 세운다
# H0 : 그룹 간의 모평균에는 차아가 없다. (요인효과는 없다)  
# H1 : 그룹 간의 모평균에는 차아가 있다. (요인효과가 있다) 

In [9]:
# 검정통계량(F값)을 계산한다
F_0 = ms_between / ms_within
print('F_0 = ', round(F_0, 2))

# 유의수준
alpha = 0.05 

# 확률변수(Random Variable)과 임계치를 구한다
rv_f = stats.f(df_between, df_within)
F_c = rv_f.ppf(1-alpha) # percent point function : 분포의 좌측 누적 확률을 인자로 하여 임계치(기각역)를 구한다

# 유의 확률(p-value)도 구해 본다
p_value = 1 - rv_f.cdf(F_0)

print('Critical Vaule = ', round(F_c, 2), ', p-value = ', round(p_value, 2))

F_0 =  19.0
Critical Vaule =  9.55 , p-value =  0.02


In [10]:
rv_f = stats.f(df_between, df_within)
alpha = 0.05 
rv_f.ppf(1-alpha)

9.552094495921155

In [11]:
# F_0 기각역에 포함되면 H0를 기각한다
# 그렇지 않으면 H0를 채택한다 (H0가 틀렸다고는 할 수 없다)
if (F_c <= F_0):
    print('F_c:', round(F_c,2), ' <= ', 'F_0:', round(F_0,2), '[H0 Rejected]')
else:
    print('F_c:', round(F_c,2), ' > ', 'F_0:', round(F_0,2), '[H0 Accepted]')

F_c: 9.55  <=  F_0: 19.0 [H0 Rejected]


In [12]:
# 결론 : 검정통계량(F_0)이 임계값(F_c)보다 크기 때문에 비료 효과는 있다고 할 수 있다.

### [예제 2] 10명 학생의 과목별 점수 (<u>주성분분석</u>)
출처 : 쿠리하라 신이치, 통계학 도감, 2019, pp. 217

In [13]:
prin_comp_anal = pd.DataFrame([[72,65,88,56,85], [65,95,80,70,80], [44,40,50,38,55], \
                               [29,34,43,24,45], [66,29,66,37,75], [51,72,43,72,65], \
                               [73,38,75,40,81], [50,71,52,63,60], [55,65,58,50,66], \
                               [42,15,46,30,50]], \
                        columns=['Korean', 'Mayh', 'English', 'Science', 'Social Studies'])
prin_comp_anal

,Korean,Mayh,English,Science,Social Studies
0,72,65,88,56,85
1,65,95,80,70,80
2,44,40,50,38,55
3,29,34,43,24,45
4,66,29,66,37,75
5,51,72,43,72,65
6,73,38,75,40,81
7,50,71,52,63,60
8,55,65,58,50,66
9,42,15,46,30,50


In [14]:
# 주성분 분석을 위해 먼저 데이터를 표준화한다

from sklearn.preprocessing import StandardScaler # 표준화 패키지 라이브러리 

# StandardScaler객체 생성
scaler = StandardScaler()

# StandardScaler 로 데이터 셋 변환 .fit( ) 과 .transform( ) 호출
scaler.fit(prin_comp_anal)
prin_comp_anal_scaled = scaler.transform(prin_comp_anal)

# transform( )시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
prin_comp_anal_scaled = pd.DataFrame(data=prin_comp_anal_scaled, columns=prin_comp_anal.columns)

prin_comp_anal_scaled

,Korean,Mayh,English,Science,Social Studies
0,1.275338,0.536856,1.805873,0.502162,1.434488
1,0.759305,1.815085,1.288060,1.380947,1.052975
2,-0.788793,-0.528334,-0.653739,-0.627703,-0.854588
3,-1.894578,-0.783980,-1.106825,-1.506487,-1.617614
4,0.833024,-0.997018,0.381887,-0.690473,0.671462
5,-0.272760,0.835109,-1.106825,1.506487,-0.091563
6,1.349057,-0.613550,0.964427,-0.502162,1.129278
7,-0.346479,0.792502,-0.524286,0.941554,-0.473076
8,0.022116,0.536856,-0.135926,0.125541,-0.015261
9,-0.936231,-1.593525,-0.912645,-1.129865,-1.236101


In [60]:
from sklearn.decomposition import PCA
pca = PCA(n_components=5) # 주성분을 몇개로 할지 결정
printcipalComponents = pca.fit_transform(prin_comp_anal_scaled)
pca_df = pd.DataFrame(data=printcipalComponents, columns = ['Comp1', 'Comp2', 'Comp3', 'Comp4', 'Comp5']) # 주성분으로 이루어진 데이터 프레임 구성
pca_df

,Comp1,Comp2,Comp3,Comp4,Comp5
0,-2.589952,-0.780563,-0.348921,0.200187,-0.082757
1,-2.708026,0.949713,-0.556514,0.029007,-0.005893
2,1.562072,0.003862,-0.107089,0.014934,0.035528
3,3.131953,0.031782,-0.689062,-0.131451,-0.114282
4,-0.297717,-1.573170,0.434086,-0.088459,-0.040894
5,-0.184206,1.899103,0.782164,-0.035038,-0.145680
6,-1.269681,-1.715993,0.280403,-0.128725,0.045093
7,0.005827,1.445271,0.102402,0.110566,0.147493
8,-0.186346,0.441400,-0.104084,-0.262646,0.117462
9,2.536076,-0.701404,0.206615,0.291624,0.043930


In [61]:
# 주성분의 표준편차, 분산, 기여율 계산
pca_std = pca_df.std(ddof=0) # ddof=0 : 모표준편차

pca_std = pca_std.values # Series to np.array

pca_variance = pow(pca_std, 2) # 주성분의 분산 (이것은 고유값과 같다)

variance_ratio = pca_variance / 5 # # 설명변수의 총 개수 : 5
# 기여율은 pca.explained_variance_ratio_ 로 구할 수도 있다.

print('Std of Principal Comps = ', pca_std)
print('기여율 of Principal Comps = ', variance_ratio)

Std of Principal Comps =  [1.85719032 1.15386121 0.43134381 0.15820671 0.09144166]
기여율 of Principal Comps =  [0.68983118 0.26627914 0.0372115  0.00500587 0.00167232]


In [91]:
# 고유값과 고유벡터 (주성분부하량, Loading)
eigenvalue = pca.explained_variance_
eigenvector = pca.components_
print('eigenvalue :', eigenvalue)
print('eigenvector :\n', eigenvector)

eigenvalue : [3.83239543 1.47932855 0.20673053 0.0278104  0.00929064]
eigenvector :
 [[-0.48754806 -0.37113797 -0.4661872  -0.37854667 -0.51373671]
 [-0.32412603  0.59975708 -0.3574677   0.59237462 -0.23778753]
 [ 0.43949561 -0.4602128  -0.6298468   0.4026948   0.19020434]
 [-0.16116201 -0.49896945  0.50757614  0.57456001 -0.37054494]
 [ 0.6618985   0.20440576  0.02336256 -0.11624292 -0.71137222]]


In [92]:
# 고유값과 고유벡터 (주성분부하량, Loading) - 2
# 다른 방법 (공분산행렬과 고유값분해)
prin_comp_anal_scaled_cov = np.dot(prin_comp_anal_scaled.values.T,prin_comp_anal_scaled.values)/9

w, v = np.linalg.eig(prin_comp_anal_scaled_cov)

print('eigenvalue :', w)
print('eigenvector :\n', v)

eigenvalue : [3.83239543 1.47932855 0.20673053 0.00929064 0.0278104 ]
eigenvector :
 [[-0.48754806 -0.32412603  0.43949561 -0.6618985   0.16116201]
 [-0.37113797  0.59975708 -0.4602128  -0.20440576  0.49896945]
 [-0.4661872  -0.3574677  -0.6298468  -0.02336256 -0.50757614]
 [-0.37854667  0.59237462  0.4026948   0.11624292 -0.57456001]
 [-0.51373671 -0.23778753  0.19020434  0.71137222  0.37054494]]


In [93]:
# 방법에 따라 결과값이 약간 다르다 ..

### [예제 3] Iris 데이터의 (<u>주성분분석</u>)
출처 : PCA(주성분 분석)_Python(파이썬) 코드 포함 (딥상어동)  
https://m.blog.naver.com/tjdrud1323/221720259834

In [46]:
# Iris 데이터
iris_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
iris_df = pd.read_csv(iris_url, names=['sepal length','sepal width','petal length','petal width','target'])
iris_df.head()

,sepal length,sepal width,petal length,petal width,target
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [47]:
# 주성분 분석을 위해 먼저 데이터를 표준화한다

#from sklearn.preprocessing import StandardScaler  # 표준화 패키지 라이브러리 
x = iris_df.drop(['target'], axis=1).values # 독립변인들의 value값만 추출
y = iris_df['target'].values # 종속변인 추출

x = StandardScaler().fit_transform(x) # x객체에 x를 표준화한 데이터를 저장

features = ['sepal length', 'sepal width', 'petal length', 'petal width']
x_df = pd.DataFrame(x, columns=features)
x_df.head()

,sepal length,sepal width,petal length,petal width
0,-0.900681,1.032057,-1.341272,-1.312977
1,-1.143017,-0.124958,-1.341272,-1.312977
2,-1.385353,0.337848,-1.398138,-1.312977
3,-1.506521,0.106445,-1.284407,-1.312977
4,-1.021849,1.263460,-1.341272,-1.312977


In [48]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2) # 주성분을 몇개로 할지 결정
printcipalComponents = pca.fit_transform(x)
pca_df = pd.DataFrame(data=printcipalComponents, columns = ['principal component1', 'principal component2']) # 주성분으로 이루어진 데이터 프레임 구성
pca_df

,principal component1,principal component2
0,-2.264542,0.505704
1,-2.086426,-0.655405
2,-2.367950,-0.318477
3,-2.304197,-0.575368
4,-2.388777,0.674767
...,...,...
145,1.870522,0.382822
146,1.558492,-0.905314
147,1.520845,0.266795
148,1.376391,1.016362


In [49]:
pca.explained_variance_ratio_

array([0.72770452, 0.23030523])

In [51]:
# 주성분의 표준편차, 분산, 기여율 계산
pca_std = pca_df.std(ddof=0) # ddof=0 : 모표준편차

pca_std = pca_std.values # Series to np.array

pca_variance = pow(pca_std, 2) # 주성분의 분산 (이것은 고유값과 같다)

variance_ratio = pca_variance / 4 # 설명변수의 총 개수 : 4
# 기여율은 pca.explained_variance_ratio_ 로 구할 수도 있다.

print('Std of Principal Comps = ', pca_std)
print('기여율 of Principal Comps = ', variance_ratio)

Std of Principal Comps =  [1.70611198 0.95980255]
기여율 of Principal Comps =  [0.72770452 0.23030523]


### [예제 4] 유방 엑스선 검사가 양성일 때 실제 유방암일 확률 (<u>베이즈 통계학-사후확률</u>)
출처 : 쿠리하라 신이치, 통계학 도감, 2019, pp. 254

#### 베이즈 정리
P(B|A) = P(A|B) ∙ P(B) / P(A) 

In [95]:
P_B = 0.4 / 100 # 사전확률 P(A) : 0.4%
P_A_B = 80 / 100 # 우도 P(A|B) : 80%
P_A = 0.32 /100 + 9.96 / 100 # 진양성 0.32%, 위양성 9.96% (전체 양성 : 10.28%)

In [97]:
P_B_A = P_A_B * P_B / P_A
print('사후확률 = ', round(P_B_A*100, 2), ' [%]')

사후확률 =  3.11  [%]
